## Table of contents
1. [Introduction](#introduction)

2. [Linear Regression](#linreg)

    2.1 [Preprocessing](#preprocessing)
    
    2.2 [Algorithm](#algorithm)
    
    2.3 [Predictions](#predictions)

3. [Neural networks](#neural)
    
    2.1 [Algorithm](#algorithm2)
    
    2.2 [Predictions](#predictions2)
    
4. [References](#references)

## Introduction
The goal of this project is to make 2 predictive models which predict wind turbine power from wind turbine speed. The dataset features and labels (i.e. input and output values) [1] https://medium.com/technology-nineleaps/some-key-machine-learning-definitions-b524eb6cb48 that are both continuous so regression models are employed. The two models are decision tree regression and neural networks respectively.

## Decision Tree Regression
This project will use the the DecisionTreeRegressor function from the sklearn package to perform regression using Decision Trees 

Decision Trees work by breaking down the dataset into smaller and smaller segments while a "decision tree" (i.e. a node structure with tests at each node to divide the data) is developed node by node.[1] https://www.saedsayad.com/decision_tree_reg.html

This algorithm works in the same way for both classification and regression.


### Preprocessing 
First the data is imported and preprocessed. The preprocessing consists in splitting up the data into feature data (x values) and labels (y values) and rehsaping the X values as the LinearRegression() from linear_model function does not take a 1D array for the X values.

In [1]:
# import data 
import pandas as pd
lin_data = pd.read_csv('powerproduction.csv')
lin_data.head()

# X and y values for regression
X = lin_data.iloc[:, 0].values
y = lin_data.iloc[:, 1].values

# The X values are reshaped as 
# they only contain one feature
X = X.reshape(-1, 1)

Preprocessing often involves scaling the data and removing outliers etc. The data will not be scaled in this project so the models are as simple as possible and there is no confusion between the predicted data and the observed data. Outliers will not be removed as there is no specific reason given why they are not accurate data.

### Algorithm
The train_test_split function is defined using the model_selection module from the sklearn package. This is used to randomly split the data into training and testing data.
The DecisionTreeRegressor function is defined and used to fit a regressor to the X_train and y_train datasets.

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

regressor = DecisionTreeRegressor();
regressor.fit(X_train, y_train);

### Predictions
The predicted values are calculated using the regressor and the X_test testing data. Then various metrics (mean absolute error, mean squared error and root mean squared error) are calculated to test the efficacy of the model. In addition, the coefficient of variation (the root mean squared error RMSE as a percentage of the mean of the observed valeus) is calculated.

In [3]:
from sklearn import metrics
import numpy as np
y_pred = regressor.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('Mean of observed y values:', np.mean(y))
# coefficient of variation 
print('Coefficient of variation:', (100*np.sqrt(metrics.mean_squared_error(y_test, y_pred)))/np.mean(y))

Mean Absolute Error: 6.4924
Mean Squared Error: 225.193058
Root Mean Squared Error: 15.006433886836673
Mean of observed y values: 48.014584
Coefficient of variation: 31.253907951877025


The coefficient of variation is 31.25%. A good coefficient of variation is considered to be less than 25% [3]. https://www.kw-engineering.com/how-to-assess-a-regressions-predictive-power-energy-use/ so the decision tree regression model is not very accurate at predicting power.

## Neural Networks 
This project will use the the  function from the sklearn package to perform regression using Artificial Neural Networks.

Neural networks operate in a similar manner to neurons in the brain [4]. Nodes in the neural network are connected together and they each have an activation function which depends on the inputs (which are the outputs of the other neurons, obtained via the connections). 

Activation functions differ but generally they are approximately 1 when a certain threshold value has been reached and approximately 0 if that value has not been reached. In addition, the connections between the neurons have weights which amplify or diminish the outputs (in an artificial neural network these are  set to randomly chosen small numbers [5] https://machinelearningmastery.com/why-initialize-a-neural-network-with-random-weights/

In a typical artificial neural network, there is an input layer of neurons (which have inputs not connected to any other neuron), an output layer (which have outputs not connected to any other neuron) and a hidden layer (neurons which have inputs and outputs connected to other neurons). A deep neural network has multiple hidden layers. [6] https://machinelearningmastery.com/what-is-deep-learning/

Neural networks can be used to learn functions and patterns to make predictions. Tensorflow makes predictions using neural networks.

### Algorithm
The data preprocessed for the decision tree classifier is used again for this.

The train_test_split function is defined using the model_selection module from the sklearn package. This is used to randomly split the data into training and testing data.

The Input and Dense classes are imported from tensorflow.keras.layers. The input layer and the hidden layers of the neural network are created as objects of these classes.[7] https://stackabuse.com/tensorflow-2-0-solving-classification-and-regression-problems/.

There are 500 nodes in the hidden layer of the artificial neural network (so this is not a deep neural network as there is only one hidden layer). This is because there are 500 features and 500 labels to train the model on.

The activation function in this case is the rectified linear activation function or "relu". It is a piecewise linear function that returns the input if the input is positive. If the input is not positive it outputs 0. [8] https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/

A regression model called model is created using the Model class. It uses the mean_squared error function for the losos function and the Adam algorithm for the optimiser. [9] https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam

The model is compiled as follows:

In [48]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

input_layer = Input(shape=(X.shape[1],))
dense_layer_1 = Dense(5000, activation='relu')(input_layer)
#dense_layer_2 = Dense(50, activation='relu')(dense_layer_1)
#dense_layer_3 = Dense(25, activation='relu')(dense_layer_2)
output = Dense(1)(dense_layer_1)

model = Model(inputs=input_layer, outputs=output)
model.compile(loss="mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])

The model is trained on the training data as follows:

In [49]:
history = model.fit(X_train, y_train, batch_size=2, epochs=100, verbose=1, validation_split=0.2)

Epoch 1/100
160/160 [==============================] - 1s 2ms/step - loss: 1448.1001 - mean_squared_error: 1448.1001 - val_loss: 462.6766 - val_mean_squared_error: 462.6766
Epoch 2/100
160/160 [==============================] - 0s 1ms/step - loss: 350.8300 - mean_squared_error: 350.8300 - val_loss: 500.7087 - val_mean_squared_error: 500.7087
Epoch 3/100
160/160 [==============================] - 0s 1ms/step - loss: 588.5197 - mean_squared_error: 588.5197 - val_loss: 425.1378 - val_mean_squared_error: 425.1378
Epoch 4/100
160/160 [==============================] - 0s 1ms/step - loss: 501.9128 - mean_squared_error: 501.9128 - val_loss: 433.3857 - val_mean_squared_error: 433.3857
Epoch 5/100
160/160 [==============================] - 0s 1ms/step - loss: 509.5452 - mean_squared_error: 509.5452 - val_loss: 409.3713 - val_mean_squared_error: 409.3713
Epoch 6/100
160/160 [==============================] - 0s 1ms/step - loss: 369.6604 - mean_squared_error: 369.6604 - val_loss: 430.7142 - val_m

Epoch 49/100
160/160 [==============================] - 0s 1ms/step - loss: 396.7653 - mean_squared_error: 396.7653 - val_loss: 488.0913 - val_mean_squared_error: 488.0913
Epoch 50/100
160/160 [==============================] - 0s 1ms/step - loss: 530.5753 - mean_squared_error: 530.5753 - val_loss: 383.0271 - val_mean_squared_error: 383.0271
Epoch 51/100
160/160 [==============================] - 0s 1ms/step - loss: 378.0256 - mean_squared_error: 378.0256 - val_loss: 371.9850 - val_mean_squared_error: 371.9850
Epoch 52/100
160/160 [==============================] - 0s 1ms/step - loss: 441.2135 - mean_squared_error: 441.2135 - val_loss: 375.7854 - val_mean_squared_error: 375.7854
Epoch 53/100
160/160 [==============================] - 0s 1ms/step - loss: 380.0902 - mean_squared_error: 380.0902 - val_loss: 374.8144 - val_mean_squared_error: 374.8144
Epoch 54/100
160/160 [==============================] - 0s 1ms/step - loss: 470.0527 - mean_squared_error: 470.0527 - val_loss: 435.3506 - v

160/160 [==============================] - 0s 1ms/step - loss: 467.7923 - mean_squared_error: 467.7923 - val_loss: 375.1318 - val_mean_squared_error: 375.1318
Epoch 97/100
160/160 [==============================] - 0s 1ms/step - loss: 438.1107 - mean_squared_error: 438.1107 - val_loss: 371.1527 - val_mean_squared_error: 371.1527
Epoch 98/100
160/160 [==============================] - 0s 1ms/step - loss: 381.6895 - mean_squared_error: 381.6895 - val_loss: 372.8512 - val_mean_squared_error: 372.8512
Epoch 99/100
160/160 [==============================] - 0s 1ms/step - loss: 400.9613 - mean_squared_error: 400.9613 - val_loss: 369.8664 - val_mean_squared_error: 369.8664
Epoch 100/100
160/160 [==============================] - 0s 1ms/step - loss: 537.8495 - mean_squared_error: 537.8495 - val_loss: 369.3985 - val_mean_squared_error: 369.3985


### Predictions
The predicted values are calculated using the regressor and the X_test testing data as before. Then various metrics (including the coefficient of variation) are calculated to test the efficacy of the model as before.

In [50]:
y_pred = model.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('Mean of observed y values:', np.mean(y))
# coefficient of variation 
print('Coefficient of variation:', (100*np.sqrt(metrics.mean_squared_error(y_test, y_pred)))/np.mean(y))

Mean Absolute Error: 13.807355700556041
Mean Squared Error: 475.0402797605675
Root Mean Squared Error: 21.79541877919687
Mean of observed y values: 48.014584
Coefficient of variation: 45.393330449758494
